In [2]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
import pickle
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time

plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)
solvers.options['show_progress'] = False
solvers.options['abstol'] = 1e-6
solvers.options['reltol'] = 1e-6
solvers.options['feastol'] = 1e-6

# Parametri default

In [3]:
seed = 1234
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [4]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1, 0.5, 1, 2, 5, 10]
problem_sizes = [300,400,500,600]
alphas = [0.1, 0.5, 1, 2, 5]
n_prob = 10

# Data Generation Functions

In [4]:
def generate_epsilons_data(feat, targ, size, u, epsilons, n, startdir='./'):
    """
    
    """
    if not startdir.endswith("/"):
        startdir += "/"
        
    u_name = str(u).replace(".","")
    
    for e in epsilons:
        eps_name = str(e).replace(".","-")
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS", exist_ok=True)
    
    os.makedirs(startdir + "RANDSTATES_EPS", exist_ok=True)
    
    for i in range(n):
        # to generate the problem on different machines and not have the name conflicting
        i = i + 10
        
        # saving the random state in case we want to rebuild a specific problem
        rs = np.random.get_state()
        with open(f"{startdir}RANDSTATES_EPS/{size}_{i}.pkl", "wb") as p:       
            pickle.dump(rs, p)
        
        # sampling the problem and solving it with different values of epsilon
        K, target = sample_transform_problem(feat, targ, size)
        
        for e in epsilons:
            print(f"{i=} , epsilon = {e}")
            eps_name = str(e).replace(".","-")
                
            print(f"\t\tGOLDSTEIN with {e=}")
             
            ds = DataStorer()
           
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/GOLD/{i}.csv")
            
            print(f"\t\tROSEN with {e=}")
            ds = DataStorer()

            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            
            ds.save_to_file(f"{startdir}C{u_name}EPS_{eps_name}/{size}/ROS/{i}.csv")
            print("\t\tEXTERNAL SOLVER")
            
            sol = solve_with_cvxopt(K, target, e, u)['dual objective']
            
            with open(f"{startdir}C{u_name}EPS_{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol) + "\n")

            

In [5]:
def generate_upperbounds_data(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    """
    
    """
    
    if not startdir.endswith("/"):
        startdir += "/"
    
    eps_name = str(epsilon).replace(".", "")
    
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
    
    os.makedirs(startdir + "RANDSTATES_C", exist_ok=True)    
    
    for i in range(n):
        # to generate the problem on different machines and not have the name conflicting
        i = i + 10
        
        # saving the random state in case we want to rebuild a specific problem
        rs = np.random.get_state()
        with open(f"{startdir}RANDSTATES_C/{size}_{i}.pkl", "wb") as p:       
            pickle.dump(rs, p)
        
        # sampling the problem and solving it with different values of epsilon
        K, target = sample_transform_problem(feat, targ, size)
        
        for u in upper_bounds:
            print(f"{i=} , C = {u}")
            u_name = str(u).replace(".","")
            
            print(f"\t\tGOLDSTEIN with C = {u}")
             
            ds = DataStorer()
            
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            
            
            print(f"\t\tROSEN with C = {u}")
            ds = DataStorer()
            
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C_{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            
            print("\t\tEXTERNAL SOLVER")
            sol = solve_with_cvxopt(K, target, epsilon, u)['dual objective']
            
            with open(f"{startdir}C_{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(sol)+"\n")

In [6]:
def generate_alphas_data(feat, targ, size, alphas, n=n_prob, startdir='./', maxiter=1000):
    """experiments on the alpha paramerer."""
    
    if not startdir.endswith("/"):
        startdir += "/"
    
    for alpha in alphas:
        alpha_name = str(alpha).replace(".","")
        os.makedirs(f"{startdir}{alpha_name}", exist_ok=True)
        print(f"creating {startdir}{alpha_name}")
    
    os.makedirs(f"{startdir}RANDSTATES_ALPHA", exist_ok=True)
    
    
    for i in range(n):
        
        i = i + 10
        
        # saving the random state in case we want to rebuild a specific problem
        rs = np.random.get_state()
        with open(f"{startdir}RANDSTATES_ALPHA/{size}_{i}.pkl", "wb") as p:       
            pickle.dump(rs, p)
        
        K, target = sample_transform_problem(feat, targ, size)

        sol = solve_with_cvxopt(K, target, 0.1, 1)['dual objective']
        
        with open(f"{startdir}optimal_values.txt", "a") as f:
            f.write(str(sol) + "\n")
        
        print(f"Problem {i}")
        for alpha in alphas:
            print(f"\t\t{alpha=}", end="\t")                    
            ds = DataStorer()
            
            sg, xg, vg, itg = SLBQP(K, target, 1, 0.1, eps=1e-3, maxIter=maxiter, alpha=alpha, prj_type=GOLDSTEIN, ds=ds)          
            
            alpha_name = str(alpha).replace(".","")
            ds.save_to_file(f"{startdir}{alpha_name}/{i}.csv")
            
            print(f"{itg=}")
            

# ML DATA GENERATION

In [7]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1
ROOT="./RUNS2/ML/"
avg_target_ml = sum(targ) / len(targ)

if avg_target_ml < 1e-5:
    avg_target_ml = 1
    
epsilons_ml = [avg_target_ml * i for i in [0.01, 0.05, 0.1, 0.2]]

loading from: 
/Users/gerardozinno/repos/ProgettoML/cm/data/ML-CUP19-TR.csv


In [8]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_epsilons_data(feat, targ, size, 1, epsilons_ml, n_prob, startdir=ROOT)

size=300
*************************
i=10 , epsilon = 0.17168133141590286
		GOLDSTEIN with e=0.17168133141590286
		ROSEN with e=0.17168133141590286
		EXTERNAL SOLVER
i=10 , epsilon = 0.8584066570795144
		GOLDSTEIN with e=0.8584066570795144
		ROSEN with e=0.8584066570795144
		EXTERNAL SOLVER
i=10 , epsilon = 1.7168133141590287
		GOLDSTEIN with e=1.7168133141590287
		ROSEN with e=1.7168133141590287
		EXTERNAL SOLVER
i=10 , epsilon = 3.4336266283180574
		GOLDSTEIN with e=3.4336266283180574
		ROSEN with e=3.4336266283180574


KeyboardInterrupt: 

In [ ]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_upperbounds_data(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
ml_alpha_startdir = f"RUNS/ML/ALPHA/"
generate_alphas_data(feat, targ, 500, alphas,n_prob, ml_alpha_startdir)

# CALIF DATA GENERATION

In [ ]:
feat, targ = load_california_dataset()
ROOT="./RUNS2/CAL/"
avg_target_cal = sum(targ) / len(targ)
if avg_target_cal < 1e-5:
    avg_target_cal = 1
epsilons_cal = [avg_target_cal * i for i in [0.01, 0.05, 0.1, 0.2]]

In [ ]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_epsilons_data(feat, targ, size, 1, epsilons_cal, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_upperbounds_data(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
cal_alpha_startdir = f"RUNS/CAL/ALPHA/"
generate_alphas_data(feat, targ, 500, alphas,n_prob, cal_alpha_startdir)

# AIRF DATA GENERATION

In [ ]:
feat, targ = load_airfoil_dataset()
ROOT="./RUNS2/AIRF/"
avg_target_airf = sum(targ) / len(targ)
if avg_target_airf < 1e-5:
    avg_target_airf = 1
epsilons_airf = [avg_target_airf * i for i in [0.01, 0.05, 0.1, 0.2]]

In [ ]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_epsilons_data(feat, targ, size, 1, epsilons_airf, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    print(f"{size=}\n*************************")
    generate_upperbounds_data(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
airf_alpha_startdir = f"RUNS/AIRF/ALPHA/"
generate_alphas_data(feat, targ, 500, alphas,n_prob, airf_alpha_startdir)